### 필수과제1. 임계값 조절하여 분류 테스트 변화 확인해 보기
- 임계값을 3개 정도(원하는 값으로) 잡고 각각의 평가 지표 변화 확인해 보기!

In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import Binarizer

In [2]:
# 타이타닉 데이터셋 이용
titanic_df = sns.load_dataset("titanic")

In [3]:
titanic_df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [4]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [5]:
# 결측치 확인
titanic_df.isna().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [6]:
# deck 제거
titanic_df.drop("deck", axis=1, inplace=True)

In [7]:
# age가 결측치인 행 제거
titanic_df = titanic_df.dropna()
titanic_df.reset_index(inplace=True)
titanic_df.drop("index", axis=1, inplace=True)

C:\Users\doram\AppData\Local\Temp\ipykernel_9812\2859378338.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  titanic_df.drop("index", axis=1, inplace=True)


In [8]:
titanic_df.isna().sum()

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alive          0
alone          0
dtype: int64

In [11]:
# 수치형 자료만 남기고 제고
titanic_df = titanic_df.select_dtypes(exclude=["object","category"])

In [12]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   survived    712 non-null    int64  
 1   pclass      712 non-null    int64  
 2   age         712 non-null    float64
 3   sibsp       712 non-null    int64  
 4   parch       712 non-null    int64  
 5   fare        712 non-null    float64
 6   adult_male  712 non-null    bool   
 7   alone       712 non-null    bool   
dtypes: bool(2), float64(2), int64(4)
memory usage: 34.9 KB


In [13]:
df_X = titanic_df.drop("survived", axis=1)
df_y = titanic_df["survived"]

In [14]:
df_X

,pclass,age,sibsp,parch,fare,adult_male,alone
0,3,22.0,1,0,7.2500,True,False
1,1,38.0,1,0,71.2833,False,False
2,3,26.0,0,0,7.9250,False,True
3,1,35.0,1,0,53.1000,False,False
4,3,35.0,0,0,8.0500,True,True
...,...,...,...,...,...,...,...
707,3,39.0,0,5,29.1250,False,False
708,2,27.0,0,0,13.0000,True,True
709,1,19.0,0,0,30.0000,False,True
710,1,26.0,0,0,30.0000,True,True


In [15]:
df_y

0      0
1      1
2      1
3      1
4      0
      ..
707    0
708    0
709    1
710    1
711    0
Name: survived, Length: 712, dtype: int64

In [16]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.3, random_state=111)

In [17]:
lr = LogisticRegression()

In [18]:
lr.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [19]:
# 예측 후 값 추출
pred = lr.predict(X_test)

In [20]:
pred

array([0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0], dtype=int64)

In [28]:
# 테스트 결과 (default)
confusion = confusion_matrix(y_test, pred)
accuracy = round(accuracy_score(y_test, pred), 3)
precision = round(precision_score(y_test, pred), 3)
recall = round(recall_score(y_test, pred), 3)
f1 = round(f1_score(y_test, pred), 3)

print("confusion matrix")
print(confusion)
print("정확도: {}, 정밀도: {}, 재현율: {}, F1: {}".format(accuracy, precision, recall, f1))

confusion matrix
[[116  11]
 [ 18  69]]
정확도: 0.864, 정밀도: 0.862, 재현율: 0.793, F1: 0.826


In [32]:
# 예측 확률 추출
pred_p = lr.predict_proba(X_test)
pred_p_sel = np.concatenate([pred_p, pred.reshape(-1,1)], axis=1)

In [42]:
# threshold = 0.1~0.9일 때의 순차적 결과
vals = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
for val in vals:
    binarizer = Binarizer(threshold=val)
    pred = binarizer.fit_transform(pred_p)[:,1]
    print("threshold: {}".format(val))

    confusion = confusion_matrix(y_test, pred)
    accuracy = round(accuracy_score(y_test, pred), 3)
    precision = round(precision_score(y_test, pred), 3)
    recall = round(recall_score(y_test, pred), 3)
    f1 = round(f1_score(y_test, pred), 3)

    print("confusion matrix")
    print(confusion)
    print("정확도: {}, 정밀도: {}, 재현율: {}, F1: {}".format(accuracy, precision, recall, f1))
    print()

threshold: 0.1
confusion matrix
[[ 24 103]
 [  0  87]]
정확도: 0.519, 정밀도: 0.458, 재현율: 1.0, F1: 0.628

threshold: 0.2
confusion matrix
[[84 43]
 [ 8 79]]
정확도: 0.762, 정밀도: 0.648, 재현율: 0.908, F1: 0.756

threshold: 0.3
confusion matrix
[[104  23]
 [  9  78]]
정확도: 0.85, 정밀도: 0.772, 재현율: 0.897, F1: 0.83

threshold: 0.4
confusion matrix
[[112  15]
 [ 16  71]]
정확도: 0.855, 정밀도: 0.826, 재현율: 0.816, F1: 0.821

threshold: 0.5
confusion matrix
[[116  11]
 [ 18  69]]
정확도: 0.864, 정밀도: 0.862, 재현율: 0.793, F1: 0.826

threshold: 0.6
confusion matrix
[[118   9]
 [ 24  63]]
정확도: 0.846, 정밀도: 0.875, 재현율: 0.724, F1: 0.792

threshold: 0.7
confusion matrix
[[123   4]
 [ 33  54]]
정확도: 0.827, 정밀도: 0.931, 재현율: 0.621, F1: 0.745

threshold: 0.8
confusion matrix
[[125   2]
 [ 48  39]]
정확도: 0.766, 정밀도: 0.951, 재현율: 0.448, F1: 0.609

threshold: 0.9
confusion matrix
[[127   0]
 [ 72  15]]
정확도: 0.664, 정밀도: 1.0, 재현율: 0.172, F1: 0.294



> - threshold : 0.1, 재현율(recall) 최대 (1)
> - threshold : 0.3, F1(f1_score) 최대 (0.83)
> - threshold : 0.5, 정확도(accuracy) 최대 (0.864)
> - threshold : 0.9, 정밀도(precision) 최대 (1)

- threshold값이 클수록 정밀도(precision)가 커지고 재현율(recall)이 작아진다.
- threshold가 0.3일 때 F1값이 가장 크고, 이때 정밀도와 재현율이 균형에 맞다고 볼 수 있다.